In [ ]:
import os
import re
import pandas as pd
from lib.dataframe_to_table import make_latex_table
from lib.dataframe_to_table import make_html_table
from textwrap import dedent
from collections import namedtuple
import glob

from lib.utils import DATA_DIR,TABLES_DIR,OUTPUT_DIR,TABLES_INDEX

with open(f"{DATA_DIR}/outcomes.md","r") as f:
    outcomes_md = f.read()


os.makedirs(OUTPUT_DIR,exist_ok=True)

def make_table_dict():
    """
    get tables dict[id,content] of tex and html
    """
    table_index = pd.read_csv(f'{TABLES_INDEX}',encoding="utf_8_sig").fillna("")
    Tables = namedtuple('Tables', ('tex', 'html'))

    tex_dict = {}
    html_dict = {}
    for row in table_index.itertuples():
        table = pd.read_csv(f"{TABLES_DIR}/{row.id}.csv")
        table = table.loc[:,re.split(r" *, *",row.列)]

        tex_dict[row.id]=make_latex_table(table,
            label=row.id,
            group_rows=True,
            caption=row.表名,
            layout=row.レイアウト)
        html_dict[row.id]=make_html_table(table,group_rows=True)
    return Tables(tex_dict,html_dict)

table_ids = []
def ditect_table_fn(reg):
    global table_ids
    s=reg.group(1)
    table_ids.append(s)

re.sub(r"\[@tbl:([^\]]+)\]",ditect_table_fn,outcomes_md)
table_ids = list(dict.fromkeys(table_ids))
table_dicts= make_table_dict()
latex_tables = "\n\n".join(list(map(lambda id:table_dicts.tex[id],table_ids)))
html_tables = "\n\n".join(list(map(lambda id:table_dicts.html[id],table_ids)))

with open(f"{OUTPUT_DIR}/tables.tex","w") as f:
    f.write(latex_tables)

with open(f"{OUTPUT_DIR}/tables.html","w") as f:
    f.write(html_tables)

